## Generating fixtures to test the Equation Of Time functions, the Longitude Fitting Functions and the Longitude Calculation Functions

### Expected Longitude Output is downloaded directly from https://maps.nrel.gov/pvdaq/ PVDAQ Contributed Sites

### Expected longitude = -76.6636

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
# PVInsight Code Imports
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data
path = Path.cwd().parent.parent.parent
os.chdir(path)

### Load data table from external source

#### For today's example, we're loading data from NREL's PVDAQ API, which is a publically available PV generatation data set.

In [3]:
data_frame = get_pvdaq_data(sysid=1199, year=[2015, 2016, 2017], api_key='DEMO_KEY')

CAUTION: Multiple scan rates detected!=======================] 100.0% ...queries complete in 23.7 seconds       
Scan rates (in seconds): [300, 280]
0 transitions detected.
Suggest splitting data set between:




In [4]:
gmt_offset = -5

### Running the  DataHandler

In [5]:
dh = DataHandler(data_frame[0])
dh.fix_dst()

In [6]:
dh.run_pipeline(power_col='ac_power', fix_shifts=False, correct_tz=False)

total time: 16.08 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              5.35s
Cleaning                   0.30s
Filtering/Summarizing      10.42s
    Data quality           0.42s
    Clear day detect       0.30s
    Clipping detect        8.31s
    Capacity change detect 1.39s



In [7]:
days = dh.daily_flags.no_errors

In [8]:
day_of_year = dh.day_index.dayofyear

In [9]:
day_range = np.ones(day_of_year.shape, dtype=bool)

In [10]:
daily_meas = dh.filled_data_matrix.shape[0]

In [11]:
daytime_threshold = None

In [12]:
from pvsystemprofiler.algorithms.performance_model_estimation import find_fit_costheta

In [13]:
scale_factor_costheta, costheta_fit = find_fit_costheta(dh.filled_data_matrix, dh.daily_flags.no_errors)

In [14]:
from pvsystemprofiler.algorithms.tilt_azimuth.daytime_threshold_quantile import filter_data

In [15]:
from pvsystemprofiler.algorithms.latitude.hours_daylight import calculate_hours_daylight

In [16]:
from solardatatools.algorithms import SunriseSunset

In [17]:
ss = SunriseSunset()

In [18]:
ss.run_optimizer(data=dh.filled_data_matrix)

In [19]:
hours_daylight = ss.sunset_estimates - ss.sunrise_estimates

In [20]:
#hours_daylight = calculate_hours_daylight(dh.filled_data_matrix, threshold=daytime_threshold)

In [21]:
boolean_filter = filter_data(dh.filled_data_matrix, daytime_threshold=daytime_threshold, x1=0.9, x2=0.9)

In [22]:
boolean_filter = boolean_filter * days * day_range

In [23]:
from pvsystemprofiler.utilities.hour_angle_equation import calculate_omega

In [24]:
latitude = 39.4856
longitude = -76.6636
tilt = 180
azimuth = 20
gmt_offset = -5

In [25]:
omega = calculate_omega(dh.data_sampling, dh.num_days, longitude, day_of_year, gmt_offset)

In [26]:
from pvsystemprofiler.utilities.declination_equation import delta_cooper

In [27]:
delta = delta_cooper(day_of_year, daily_meas)

In [28]:
delta_f = delta[boolean_filter]

In [29]:
omega_f = omega[boolean_filter]

In [30]:
from pvsystemprofiler.algorithms.angle_of_incidence.lambda_functions import select_function

In [31]:
func_customized, bounds = select_function(latitude, None, None)

In [32]:
from pvsystemprofiler.algorithms.angle_of_incidence.curve_fitting import run_curve_fit

In [33]:
estimates = run_curve_fit(func=func_customized, keys=['tilt', 'azimuth'], delta=delta_f, omega=omega_f,
                                  costheta=costheta_fit, boolean_filter=boolean_filter, init_values=[30, 16],
                                  fit_bounds=bounds)

In [34]:
estimates

array([31.73741817,  1.65643241])

In [35]:
from pvsystemprofiler.estimator import ConfigurationEstimator

In [36]:
est = ConfigurationEstimator(dh, gmt_offset=-5)

In [37]:
est.estimate_orientation(longitude=longitude, latitude=latitude)

In [38]:
est.tilt

31.737661653072788

In [39]:
est.azimuth

1.6557372212656074